### scrapy
- 비동기 방식으로 데이터를 수집 > 데이터 수집 속도가 빠르다.

In [1]:
import scrapy, requests
from scrapy.http import TextResponse

In [ ]:
# Gmarket 베스트 200 상품 데이터 수집

In [2]:
# 1. scrapy 프로젝트 생성

In [5]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\AIVLESCHOOLPythonClass\python_crawling\Code\3\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [9]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 0000001E 7C6C:FC35입니다.
C:\USERS\USER\AIVLESCHOOLPYTHONCLASS\PYTHON_CRAWLING\CODE\3\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [ ]:
# items.py : 수집할 데이터의 구조 정의
# middlewares.py : 데이터를 수집할 때 headers정보와 같은 내용을 설정
# pipelines.py : 데이터를 수집한 후에 코드 실행 정의
# settings.py : 크롤링 설정 : 크롤링 시간 텀, robots.txt(규칙을 지킬지) 같은 걸 설정
# spiders : 디렉토리 : 크롤링 절차를 정의 (어떤 순서로 크롤링을 할지)

In [3]:
# 2. xpath 찾기 : 링크, 상세 페이지의 데이터

In [11]:
url = "http://corners.gmarket.co.kr/Bestsellers"

In [10]:
# 링크 데이터 수집

In [12]:
request = requests.get("http://corners.gmarket.co.kr/Bestsellers")
response = TextResponse(request.url, body=request.text, encoding = 'utf-8')
#TextResponse는 가져온 웹페이지를 돔구조로 만드는데 사용됩니다 (xpath를 사용할 수 있게 도와줌)

In [14]:
#링크들을 모두 가져왔음
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract() #속성값 쓸 때는 @
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953125654620877',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953125654620877']

In [ ]:
# 상세 페이지 수집

In [17]:
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding = 'utf-8')

<200 http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953125654620877>

In [19]:
#xpath사용하면 항상 결과가 list
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract() 
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title, price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [ ]:
# 3. items.py : 코드 작성 : model(데이터의 구조, ai의 모델이 아님, 수집할 데이터의 컬럼을 정의함)

In [20]:
%%writefile gmarket/gmarket/items.py

import scrapy

#scrapy.Item의 상속을 받음
class GmarketItem(scrapy.Item) :
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [4]:
# 4. spider.py : 코드 작성 : 핵심이 되는 부분, 크롤링 절차를 정의

In [22]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB" #spider 여러개 할 수 있는데 spider 이름 설정해준거
    allow_domain = ["gmarket.co.kr"] #도메인 설정해야됨
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"] #최초에 request, response 할 주소
    #request, response 안해도 알아서 스크래피가 해줌
    def parse(self, response): #비동기 방식
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]: #실행될 때마다 request 하면서 멈춤(비동기니까)
            yield scrapy.Request(link, callback=self.parse_content) # 이 link로 request 한 다음에 parse_content 호출
    
    def parse_content(self, response):
        item = GmarketItem() #우리가 만든 class
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item #yield로 item을 던져줌 -> 던져진 데이터가 csv로 저장되거나 파이프라인으로 저장되거나.. 

Writing gmarket/gmarket/spiders/spider.py


In [ ]:
# 5. scrapy 프로젝트 실행
# gmarket 디렉토리
# scrapy crawl GMB -o items.csv
#GMB를 해가지고 items.csv 파일로 저장할것이다.

In [ ]:
#여기서 명령어를 실행할 수는 없고 코드 code directory => terminal 메뉴

In [21]:
%pwd
#현재 working directory

'C:\\Users\\User\\AIVLESCHOOLPythonClass\\python_crawling\\Code\\3'

In [23]:
pd.read_csv("gmarket/items.csv").tail(2)

,link,price,title
18,http://item.gmarket.co.kr/Item?goodscode=25204...,"28,800",(롯데백화점)남여공용 디스럽터 테이피 테잎 _ FS1HTB1091X
19,http://item.gmarket.co.kr/Item?goodscode=15870...,"15,900",청정우 양지 300gX3팩


In [ ]:
#settings.py 에서 ROBOTSTXT_OBEY=True 이면 robots.txt를 따르면서 크롤링한다는 것
#네이버 같은 경우 이걸 true라고 하면 전체 사용자한테 disable이기 때문에
#크롤링을 못함
#그래서 네이버 사이트의 경우 false로 설정해야 함